In [1]:
%load_ext autoreload
%autoreload 2 # autoreload all modules

In [2]:
%matplotlib inline
import sys
import pandas as pd
import numpy as np
import sciunit

from sciunit.models.examples import ConstModel # model always predicts a fixed const
from sciunit.capabilities import ProducesNumber

sys.path.append("../")

In [3]:
from src.ldmunit.models.poisson import PoissonModel
from src.ldmunit.models.rw import RWModel
from src.ldmunit.scores import MSEScore
from src.ldmunit.scores import NLLScore
from src.ldmunit.scores import NLLRWScore
from src.ldmunit.capabilities import ProducesLoglikelihood

In [4]:
class NLLRWTest(sciunit.Test):

    required_capabilities = (ProducesLoglikelihood, )
    score_type = NLLRWScore
    
    def generate_prediction(self, model):
        return model.produce_loglikelihood()
    
    def compute_score(self, observation, prediction):
        score = NLLRWScore.compute(observation, prediction)
        return score

In [5]:
raw = pd.read_csv("/Users/holliswang/Desktop/sel_trial_data.csv", header=0)
US = {'LH': np.array([1,0,0]), 'M': np.array([0,1,0]), 'HL': np.array([0,0,1])}
CS = {'m': 0, 'p': 1}
sub_1 = raw[raw['sub_id'] == 'LCP001'].iloc[:,[4,5,6]]
sub_1['stimulus_trans'] = sub_1['stimulus'].map(US)
sub_1['outcome_trans'] = sub_1['outcome'].map(CS)

# prepare data
rw_0 = RWModel({'alpha': 0.5, 'beta': 0.5}, name="RW model ($\alpha=0.5, \beta=0.5$)")
sub_1_obs = {'outcome': sub_1['outcome_trans'].to_numpy(), 'stimulus': sub_1['stimulus_trans'].to_numpy()}

In [6]:
import time
test_1 = NLLRWTest(sub_1_obs)
start = time.time()
test_1.judge(rw_0)
time_e = time.time() - start
test_1.judge(rw_0)

A negative log likelihood (axis=0) [135.85020934 125.64698458 135.63120637]
A negative log likelihood (axis=0) [135.85020934 125.64698458 135.63120637]


A negative log likelihood (RW) = 397.128